<a href="https://colab.research.google.com/github/dsandeep0138/HackerEarth-Challenges/blob/master/flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsandeep97","key":"91ed93d9f57d24d723707d120074f706"}'}

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d rednivrug/flower-recognition-he

 99% 897M/905M [00:20<00:00, 53.2MB/s]
100% 905M/905M [00:20<00:00, 45.7MB/s]


In [0]:
!unzip flower-recognition-he.zip

In [1]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
traindf = pd.read_csv('data/train.csv', dtype=str)
testdf = pd.read_csv('data/test.csv', dtype=str)

In [0]:
traindf['image_id'] = traindf['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [4]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=10,
                             width_shift_range=0.1, 
                             height_shift_range=0.1)

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="./data/train/",
x_col="image_id",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(96, 96))

Found 18540 validated image filenames belonging to 102 classes.


In [5]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./data/test/",
x_col="image_id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(96, 96))

Found 2009 validated image filenames.


In [6]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 padding='same',
                 input_shape=(96, 96, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Dropout(0.2))
  
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
  
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(102, activation='softmax'))
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 94, 94, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 64)        0         
____

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='acc',
                                factor=0.5,
                                patience=3,
                                min_lr=0.00001,
                                verbose=1)

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=50,
                    verbose=1,
                    callbacks=[reduce_lr])

Epoch 1/50
579/579 [==============================] - 99s 171ms/step - loss: 3.8886 - acc: 0.1398
Epoch 2/50
579/579 [==============================] - 95s 165ms/step - loss: 2.9874 - acc: 0.2780
Epoch 3/50
579/579 [==============================] - 95s 164ms/step - loss: 2.5387 - acc: 0.3607
Epoch 4/50
579/579 [==============================] - 95s 163ms/step - loss: 2.2387 - acc: 0.4254
Epoch 5/50
579/579 [==============================] - 95s 164ms/step - loss: 1.9909 - acc: 0.4783
Epoch 6/50
579/579 [==============================] - 95s 164ms/step - loss: 1.7689 - acc: 0.5263
Epoch 7/50
579/579 [==============================] - 96s 165ms/step - loss: 1.5814 - acc: 0.5766
Epoch 8/50
579/579 [==============================] - 95s 164ms/step - loss: 1.4234 - acc: 0.6143
Epoch 9/50
579/579 [==============================] - 95s 164ms/step - loss: 1.2735 - acc: 0.6541
Epoch 10/50
579/579 [==============================] - 96s 165ms/step - loss: 1.1396 - acc: 0.6866
Epoch 11/50
579/579

In [9]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

2009/2009 [==============================] - 9s 5ms/step


In [0]:
predicted_class_indices=np.argmax(pred, axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
filenames=[f.split('.')[0] for f in filenames]

results=pd.DataFrame({"image_id":filenames,
                      "category":predictions})
results = results.sort_values(by = ['image_id'], ascending = [True])
results.to_csv("results.csv", index=False)